# MCP Exercise: Building a Multi-Server Agent

In this exercise, you'll build a LangGraph agent that integrates both filesystem and GitHub MCP servers to automate fetching repository issues and saving summaries.

In [ ]:
# Install mcp, langgraph, and langchain if needed (uncomment when running locally)
# !pip install langchain-mcp-adapters langgraph>=0.2.0 langchain-openai mcp

import asyncio
import os
from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.prebuilt import ToolNode

In [ ]:
import os
## SET YOUR OPENAI_API_KEY HERE

## Exercise Setup

Your task is to create a LangGraph agent that can:
1. Connect to both filesystem and GitHub MCP servers
2. Fetch the latest issue from a GitHub repository
3. Summarize the issue
4. Save the summary to a file using the filesystem server

Follow the steps below to complete the implementation.

In [ ]:
async def setup_graph() -> StateGraph:
    """
    Create and return a compiled LangGraph that integrates MCP tools.

    TODO: Your task is to configure the MCP server connections only.
    The LangGraph setup is already implemented for you.
    """

    # TODO: Define MCP server connections
    # Configure both filesystem and GitHub server connections
    connections = {
        # Add filesystem server configuration here
        # Add GitHub server configuration here
    }

    # Create MCP client and get tools
    client = MultiServerMCPClient(connections)
    tools = await client.get_tools()

    # Initialize ChatOpenAI model and bind tools
    model = ChatOpenAI(model="gpt-4o", temperature=0)
    model_with_tools = model.bind_tools(tools)

    # Create ToolNode for executing tool calls
    tool_node = ToolNode(tools)

    # Define transition function for graph routing
    def should_continue(state: MessagesState):
        last = state["messages"][-1]
        if last.tool_calls:
            return "tools"
        return END

    # Define model invocation node
    async def call_model(state: MessagesState):
        messages = state["messages"]
        response = await model_with_tools.ainvoke(messages)
        return {"messages": [response]}

    # Build the state graph
    builder = StateGraph(MessagesState)
    builder.add_node("call_model", call_model)
    builder.add_node("tools", tool_node)
    builder.add_edge(START, "call_model")
    builder.add_conditional_edges("call_model", should_continue)
    builder.add_edge("tools", "call_model")

    return builder.compile()

## Instructions for Implementation

Your task is to configure the MCP server connections. The LangGraph implementation is already provided.

### Your Task: Configure MCP Server Connections

In the `connections` dictionary, you need to add two server configurations:

1. **Filesystem Server Configuration**:
2. **GitHub Server Configuration**:

### Connection Configuration Format

Each server configuration should follow this structure:
```python
"server_name": {
    "command": "command_to_run",
    "args": ["list", "of", "arguments"],
    "transport": "stdio"
}
```

Once you've configured the connections, the rest of the code will handle:
- Creating the MultiServerMCPClient
- Getting tools from both servers
- Setting up the LangGraph with model and tool nodes
- Managing the conversation flow

In [ ]:
async def run_example() -> None:
    """
    Run an example query against the graph.
    
    This function demonstrates the completed MCP integration by asking the agent
    to fetch the latest issue from langchain-ai/langgraph and save a summary.
    """
    # Set up the graph using your MCP configuration
    graph = await setup_graph()
    
    # Define the prompt for the task
    prompt = (
        "In the GitHub repository langchain-ai/langgraph (https://github.com/langchain-ai/langgraph), retrieve the most "
        "recently created issue. Provide a concise summary of the issue and "
        "append the summary to a file named 'issue_summary.txt' in the current "
        "working directory. Confirm when the summary has been saved."
    )
    
    # Invoke the graph with the prompt
    result = await graph.ainvoke({"messages": [{"role": "user", "content": prompt}]})
    
    # Print the final assistant message
    print(result["messages"][-1].content)

## Test Your Implementation

Once you've completed the functions above, run the cell below to test your implementation:

In [ ]:
# Test your implementation
await run_example()

## Expected Behavior

When working correctly, your agent should:
1. Use the GitHub server to fetch the latest issue from the langchain-ai/langgraph repository
2. Process and summarize the issue content
3. Use the filesystem server to save the summary to `issue_summary.txt`
4. Confirm that the file has been saved successfully

The final output should indicate that the task has been completed and the summary file has been created.